In [15]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [27]:
os.environ["OPENAI_API_KEY"] = getpass.getpass()
from langchain_core.globals import set_llm_cache
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [41]:
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

In [42]:
from langchain_core.prompts import ChatPromptTemplate

# system_template_old = "Given the following JSON object representing a {category} component in a Revit architectural project, give a short 1-sentence description of the component"

system_template = "You are writing a very short 1-sentence description of a component from a Revit architectural project. The following JSON object contains information about the component. The description is intended as a summary of the component for technical specifications"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{json_object}")]
)

chain = prompt_template | model

In [51]:
import json

filename = 'arch_rac_example_processed'

with open('{}.json'.format(filename), 'r') as file:
    data = json.load(file)

In [52]:
from pprint import pprint as pp

In [62]:
for cat, elems in data.items():

    for elem in elems:
        
        if "Description Human-Readable" in elem:
            # print()
            continue
        print(elem)
        response = chain.invoke({"category": cat,
                                 "json_object": elem})
        elem["Description Human-Readable"] = response.content
        # break
    # break
# print(data["Doors"][0:4])

# pp(data)

with open('{}_with_descriptions.json'.format(filename), 'w+') as file2:
        json.dump(data, file2)





{'IfcGUID': '38BWtPCUX0VgAKmmGnoLQm', 'Family and Type': 'Insitu Concrete 225mm', 'Materials': ['Concrete, Cast In Situ', 'Damp-proofing', 'Concrete, Sand/Cement Screed'], 'Elevation at Bottom': '2775.0000000000005', 'Elevation at Top': '3000.0', 'Thickness': '225.00000000000003', 'Area': '26.169000000003045', 'Volume': '5.888025000000679', 'Room Bounding': '1', 'Perimeter': '28580.00000000255', 'Category': 'Floors', 'Level': 'Level(Name=Level 2, Elevation=3000)', 'Comments': ''}
{'IfcGUID': '3t5Ey6FwDBbRFdkxqUP7z5', 'Family and Type': '3000 x 3000mm Grid', 'Materials': ['Default', 'Finishes - Interior - Plasterboard'], 'Area': '83.51144000000616', 'Volume': '4.175572000000308', 'Room Bounding': '1', 'Perimeter': '39156.000000000924', 'Height Offset From Level': '2700.0', 'Category': 'Ceilings', 'Level': 'Level(Name=Level 1, Elevation=0)', 'Comments': 'None'}
{'IfcGUID': '1LMg1ZkATBJwu7hODVWkcE', 'Family and Type': '3000 x 3000mm Grid', 'Materials': ['Default', 'Finishes - Interior - P